# 🦜🔗​ Descubriendo el Poder de ​Langchain: 
Transformación de Texto en Datos Estructurados​

# 🐛 Debug ON/OFF

## ☀️ ON

In [ ]:
import langchain

langchain.debug = True

## 🌑 OFF

In [ ]:
langchain.debug = False

# 📜 Sources

In [ ]:
from operator import itemgetter
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import re
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import PromptTemplate
import warnings

# import warning menssages that start with "UserWarning:"
warnings.filterwarnings("always", category=UserWarning, module='transformers')

class ExtractJsonOutputParser(StrOutputParser):
    def __init__(self):
        super().__init__()

    def parse_result(self, output):
        result = super().parse_result(output)
        try:
            # make a regular expression to find the content between the <json> tags
            json_regex = re.compile(r'<json>(.*)</json>', re.DOTALL)
            # find the content between the <json> tags
            json_content = json_regex.findall(result)[0]
            return json_content
        except:
            return result
        
def llama_prompt(system, human, **kwargs):
    prompt = PromptTemplate.from_template(
        template=f"<<SYS>>\n{system}\n<</SYS>>\n\n[INST]\n{human}\n[/INST]",
        partial_variables=kwargs
    )
    return prompt

system_prompt = """
You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
"""

json_format = """
<json>
{
    'highlights': array of
    {
        'order': int,
        'location': string,
        'summary': string,
    }
}
</json>
"""

def count_tokens(text: str, model) -> int:
    return len(model.pipeline.tokenizer.encode(text))

def print_memory(memory, params={}):
    print("MEMORY BUFFER==>:")
    print(memory.load_memory_variables(params)["chat_history"])
    print("<==============")

# 🦙 Cargar el Modelo: LLama-2 

In [ ]:
from sources.llama2_loader import load_llama_llm

llm = load_llama_llm()

# 🔗​ Chains

## 💬 Prompt simple: 
Resumen de un capitulo de un libro

In [ ]:
from langchain.prompts import PromptTemplate

# create prompt
prompt = PromptTemplate.from_template(
    template="Please summarize what happend in the chapter {chapter} of the book {book_title}. Be brief 20 words or less."
)

chain = prompt | llm

### invoke(): 
Ejecuta una cadena

In [ ]:
chain.invoke({"book_title": "Harry Potter and the Philosopher's Stone","chapter": "1"})

### batch(): 
Ejecuta la misma cadena varias veces con diferentes entradas

In [ ]:
chain.batch([
    {"book_title": "Harry Potter and the Philosopher's Stone","chapter": "1"},
    {"book_title": "Harry Potter and the Philosopher's Stone","chapter": "2"},
])

### stream():
Delvuelve la salida poco a poco, no es necesario que termine de generar todo el prompt (simula que escribe la ia)

In [ ]:
# No funciona
for t in chain.stream({"book_title": "Harry Potter and the Philosopher's Stone","chapter": "1"}):
    print(t)

## 💬🦜 Prompt con parser:
Genera highlights en formato JSON

### 💬Prompt

In [ ]:
from langchain.prompts import PromptTemplate

system_prompt = """
system_prompt = "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user asks. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
"""

human_prompt = """
Use your knowledge of the Harry Potter universe to list the top 5 highlights that happend in the chapter 1 {chapter} of the book {book_title}. Order the highlights in order of importance. Include the location where the highlight take place. Return the output in a JSON document with the following format, enclose the document with the tags <json> and </json>:
{format}
"""

json_format = """
<json>
{
    'highlights': array of
    {
        'order': int,
        'location': string,
        'summary': string,
    }
}
</json>
"""

prompt = PromptTemplate.from_template(
    template=f"<<SYS>>\n{system_prompt}\n<</SYS>>\n\n[INST]\n{human_prompt}\n[/INST]",
    partial_variables={'format': json_format}
)

### 🦜Parser

In [ ]:
import re
from langchain.schema.output_parser import StrOutputParser

class ExtractJsonOutputParser(StrOutputParser):
    def __init__(self):
        super().__init__()

    def parse_result(self, output):
        result = super().parse_result(output)
        try:
            # make a regular expression to find the content between the <json> tags
            json_regex = re.compile(r'<json>(.*)</json>', re.DOTALL)
            # find the content between the <json> tags
            json_content = json_regex.findall(result)[0]
            return json_content
        except:
            return result

### 🔗 Chain

In [ ]:
import json

chain = prompt | llm | ExtractJsonOutputParser() | json.loads

response = chain.invoke({
    "book_title": "Harry Potter and the Philosopher's Stone",
    "chapter": "1"
})

print(response.__class__)
display(response)

## 🔗🔗🔗 Encadenando chains:


### Secuencial
Resumen + highlights
```
🔗summary
    |
🔗highlights
```

In [ ]:
system_prompt = """
system_prompt = "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
"""

summary_prompt = """
Use your knowledge of the Harry Potter universe to summarize what happend in the chapter {chapter} of the book {book_title}. Be brief, 50 words maximum.
"""

highlights_prompt = """
Here is the summary of a chapter of a Harry Potter book:
{summary}
Based in that summary, identify and list the top 5 highlights that happend in it. Never add content that is not included in the summary. Order the highlights in order of importance. Include the location where the highlight take place. Return the output in a JSON document with the following format, enclose the document with the tags <output> and </output>:
{format}
"""

prompt_summary = llama_prompt(system_prompt, summary_prompt)
prompt_highlights = llama_prompt(system_prompt, highlights_prompt, format=json_format)

chain_summary = prompt_summary | llm | StrOutputParser()
chain_highlights = {"summary": chain_summary} | prompt_highlights | llm | ExtractJsonOutputParser()

response = chain_highlights.invoke({
    "book_title": "Harry Potter and the Philosopher's Stone",
    "chapter": "1",
})

In [ ]:
print(response)

### Flow
Polaridad de una relación
```
      🔗interactions
        /        \
       /          \
  🔗positive   🔗negative
       \           /
        \         /
         🔗polarity
```

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

interactions = (
    {'character_1': RunnablePassthrough(), 'character_2': RunnablePassthrough()}
    | llama_prompt(system_prompt, "List five things that happend between the characters {character_1} and {character_2}. Be brief, describe each interaction in 10 words maximum. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.")
    | llm
    | StrOutputParser()
    | {'interactions': RunnablePassthrough()}
)

positive = (
    llama_prompt(system_prompt, "Given the next list of interactions between two character, argue if they are positive and why: {interactions}")
    | llm
    | StrOutputParser()
)

negative = (
    llama_prompt(system_prompt, "Given the next list of interactions between two character, argue if they are negative and why: {interactions}")
    | llm
    | StrOutputParser()
)

polarity = (
    llama_prompt(system_prompt, "Your task is to measure the polarity of the relationship between two characters. Assign a value between -1 (very bad relationship) and +1 (very good relationship). Use the next context to help you: \n {positive} \n {negative}")
    | llm
    | StrOutputParser()
)

chain = (
    interactions
    | {'positive': positive, 'negative': negative}
    | polarity
)

response = chain.invoke(("Harry Potter", "Draco Malfoy"))

In [ ]:
print(response)

# 📖 Documentos

## Tamaño de una ventana de contexto de 4000 tokens

In [ ]:
ventana ="""
xml version='1.0' encoding='utf-8'?

CHAPTER  ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of
number four, Privet Drive, were proud to say that they were
perfectly normal, thank you very much. They were the last people
you’d expect to be involved in anything strange or
mysterious, because they just didn’t hold with such
nonsense.

Mr. Dursley was the director of a firm called Grunnings, which
made drills. He was a big, beefy man with hardly any neck, although
he did have a very large mustache. Mrs. Dursley was thin and blonde
and had nearly twice the usual amount of neck, which came in very
useful as she spent so much of her time craning over garden fences,
spying on the neighbors. The Dursleys had a small son called Dudley
and in their opinion there was no finer boy anywhere.

The Dursleys had everything they wanted, but they also had a
secret, and their greatest fear was that somebody would discover
it. They didn’t think they could bear it if anyone found out
about the Potters. Mrs. Potter was Mrs. Dursley’s sister, but
they hadn’t met for several years; in fact, Mrs. Dursley
pretended she didn’t have a sister, because her sister and
her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors
would say if the Potters arrived in the street. The Dursleys knew
that the Potters had a small son, too, but they had never even seen
him. This boy was another good reason for keeping the Potters away;
they didn’t want Dudley mixing with a child like that.

When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday our
story starts, there was nothing about the cloudy sky outside to
suggest that strange and mysterious things would soon be happening
all over the country. Mr. Dursley hummed as he picked out his most
boring tie for work, and Mrs. Dursley gossiped away happily as she
wrestled a screaming Dudley into his high chair.

None of them noticed a large, tawny owl flutter past the
window.

At half past eight, Mr. Dursley picked up his briefcase, pecked
Mrs. Dursley on the cheek, and tried to kiss Dudley good-bye but
missed, because Dudley was now having a tantrum and throwing his
cereal at the walls. “Little tyke,” chortled Mr.
Dursley as he left the house. He got into his car and backed out of
number four’s drive.

It was on the corner of the street
that he noticed the first sign of something peculiar — a cat
reading a map. For a second, Mr. Dursley didn’t realize what
he had seen — then he jerked his head around to look again.
There was a tabby cat standing on the corner of Privet Drive, but
there wasn’t a map in sight. What could he have been thinking
of? It must have been a trick of the light. Mr. Dursley blinked and
stared at the cat. It stared back. As Mr. Dursley drove around the
corner and up the road, he watched the cat in his mirror. It was
now reading the sign that said Privet Drive — no,
looking at the sign; cats couldn’t read maps or
signs. Mr. Dursley gave himself a little shake and put the cat out
of his mind. As he drove toward town he thought of nothing except a
large order of drills he was hoping to get that day.

But on the edge of town, drills were
driven out of his mind by something else. As he sat in the usual
morning traffic jam, he couldn’t help noticing that there
seemed to be a lot of strangely dressed people about. People in
cloaks. Mr. Dursley couldn’t bear people who dressed in funny
clothes — the getups you saw on young people! He supposed
this was some stupid new fashion. He drummed his fingers on the
steering wheel and his eyes fell on a huddle of these weirdos
standing quite close by. They were whispering excitedly together.
Mr. Dursley was enraged to see that a couple of them weren’t
young at all; why, that man had to be older than he was, and
wearing an emerald-green cloak! The nerve of him! But then it
struck Mr. Dursley that this was probably some silly stunt —
these people were obviously collecting for
something . . . yes, that would be it. The traffic
moved on and a few minutes later, Mr. Dursley arrived in the
Grunnings parking lot, his mind back on drills.

Mr. Dursley always sat with his back to the window in his office
on the ninth floor. If he hadn’t, he might have found it
harder to concentrate on drills that morning. He
didn’t see the owls swooping past in broad daylight, though
people down in the street did; they pointed and gazed open-mouthed
as owl after owl sped overhead. Most of them had never seen an owl
even at nighttime. Mr. Dursley, however, had a perfectly normal,
owl-free morning. He yelled at five different people. He made
several important telephone calls and shouted a bit more. He was in
a very good mood until lunchtime, when he thought he’d
stretch his legs and walk across the road to buy himself a bun from
the bakery.

He’d forgotten all about the people in cloaks until he
passed a group of them next to the baker’s. He eyed them
angrily as he passed. He didn’t know why, but they made him
uneasy. This bunch were whispering excitedly, too, and he
couldn’t see a single collecting tin. It was on his way back
past them, clutching a large doughnut in a bag, that he caught a
few words of what they were saying.

“The Potters, that’s right, that’s what I
heard —”

“— yes, their son, Harry —”

Mr. Dursley stopped dead. Fear flooded him. He looked back at
the whisperers as if he wanted to say something to them, but
thought better of it.

He dashed back across the road, hurried up to his office,
snapped at his secretary not to disturb him, seized his telephone,
and had almost finished dialing his home number when he changed his
mind. He put the receiver back down and stroked his mustache,
thinking . . . no, he was being stupid. Potter
wasn’t such an unusual name. He was sure there were lots of
people called Potter who had a son called Harry. Come to think of
it, he wasn’t even sure his nephew was called Harry.
He’d never even seen the boy. It might have been Harvey. Or
Harold. There was no point in worrying Mrs. Dursley; she always got
so upset at any mention of her sister. He didn’t blame her
— if he’d had a sister like
that . . . but all the same, those people in
cloaks . . .

He found it a lot harder to concentrate on drills that afternoon
and when he left the building at five o’clock, he was still
so worried that he walked straight into someone just outside the
door.

“Sorry,” he grunted, as the tiny old man stumbled
and almost fell. It was a few seconds before Mr. Dursley realized
that the man was wearing a violet cloak. He didn’t seem at
all upset at being almost knocked to the ground. On the contrary,
his face split into a wide smile and he said in a squeaky voice
that made passersby stare, “Don’t be sorry, my dear
sir, for nothing could upset me today! Rejoice, for You-Know-Who
has gone at last! Even Muggles like yourself should be celebrating,
this happy, happy day!”

And the old man hugged Mr. Dursley around the middle and walked
off.

Mr. Dursley stood rooted to the spot. He had been hugged by a
complete stranger. He also thought he had been called a Muggle,
whatever that was. He was rattled. He hurried to his car and set
off for home, hoping he was imagining things, which he had never
hoped before, because he didn’t approve of imagination.

As he pulled into the driveway of number four, the first thing
he saw — and it didn’t improve his mood — was the
tabby cat he’d spotted that morning. It was now sitting on
his garden wall. He was sure it was the same one; it had the same
markings around its eyes.

“Shoo!” said Mr. Dursley loudly.

The cat didn’t move. It just gave him a stern look. Was
this normal cat behavior? Mr. Dursley wondered. Trying to pull
himself together, he let himself into the house. He was still
determined not to mention anything to his wife.

Mrs. Dursley had had a nice, normal day. She told him over
dinner all about Mrs. Next Door’s problems with her daughter
and how Dudley had learned a new word (“Won’t!”).
Mr. Dursley tried to act normally. When Dudley had been put to bed,
he went into the living room in time to catch the last report on
the evening news:

“And finally, bird-watchers everywhere have reported that
the nation’s owls have been behaving very unusually today.
Although owls normally hunt at night and are hardly ever seen in
daylight, there have been hundreds of sightings of these birds
flying in every direction since sunrise. Experts are unable to
explain why the owls have suddenly changed their sleeping
pattern.” The newscaster allowed himself a grin. “Most
mysterious. And now, over to Jim McGuffin with the weather. Going
to be any more showers of owls tonight, Jim?”

“Well, Ted,” said the weatherman, “I
don’t know about that, but it’s not only the owls that
have been acting oddly today. Viewers as far apart as Kent,
Yorkshire, and Dundee have been phoning in to tell me that instead
of the rain I promised yesterday, they’ve had a downpour of
shooting stars! Perhaps people have been celebrating Bonfire Night
early — it’s not until next week, folks! But I can
promise a wet night tonight.”

Mr. Dursley sat frozen in his armchair. Shooting stars all over
Britain? Owls flying by daylight? Mysterious people in cloaks all
over the place? And a whisper, a whisper about the
Potters . . .

Mrs. Dursley came into the living room carrying two cups of tea.
It was no good. He’d have to say something to her. He cleared
his throat nervously. “Er — Petunia, dear — you
haven’t heard from your sister lately, have you?”

As he had expected, Mrs. Dursley looked shocked and angry. After
all, they normally pretended she didn’t have a sister.

“No,” she said sharply. “Why?”

“Funny stuff on the news,” Mr. Dursley mumbled.
“Owls . . . shooting
stars . . . and there were a lot of funny-looking
people in town today . . .”

“So?” snapped Mrs. Dursley.

“Well, I just thought . . .
maybe . . . it was something to do
with . . . you know . . . her
crowd.”

Mrs. Dursley sipped her tea through pursed lips. Mr. Dursley
wondered whether he dared tell her he’d heard the name
“Potter.” He decided he didn’t dare. Instead he
said, as casually as he could, “Their son — he’d
be about Dudley’s age now, wouldn’t he?”

“I suppose so,” said Mrs. Dursley stiffly.

“What’s his name again? Howard, isn’t
it?”

“Harry. Nasty, common name, if you ask me.”

“Oh, yes,” said Mr. Dursley, his heart sinking
horribly. “Yes, I quite agree.”

He didn’t say another word on the subject as they went
upstairs to bed. While Mrs. Dursley was in the bathroom, Mr.
Dursley crept to the bedroom window and peered down into the front
garden. The cat was still there. It was staring down Privet Drive
as though it were waiting for something.

Was he imagining things? Could all this have anything to do with
the Potters? If it did . . . if it got out that they
were related to a pair of — well, he didn’t think he
could bear it.

The Dursleys got into bed. Mrs. Dursley fell asleep quickly but
Mr. Dursley lay awake, turning it all over in his mind. His last,
comforting thought before he fell asleep was that even if the
Potters were involved, there was no reason for them to come
near him and Mrs. Dursley. The Potters knew very well what he and
Petunia thought about them and their kind. . . . He
couldn’t see how he and Petunia could get mixed up in
anything that might be going on — he yawned and turned over
— it couldn’t affect
them. . . .

How very wrong he was.

Mr. Dursley might have been drifting into an uneasy sleep, but
the cat on the wall outside was showing no sign of sleepiness. It
was sitting as still as a statue, its eyes fixed unblinkingly on
the far corner of Privet Drive. It didn’t so much as quiver
when a car door slammed on the next street, nor when two owls
swooped overhead. In fact, it was nearly midnight before the cat
moved at all.

A man appeared on the corner the cat
had been watching, appeared so suddenly and silently you’d
have thought he’d just popped out of the ground. The
cat’s tail twitched and its eyes narrowed.

Nothing like this man had ever been seen on Privet Drive. He was
tall, thin, and very old, judging by the silver of his hair and
beard, which were both long enough to tuck into his belt. He was
wearing long robes, a purple cloak that swept the ground, and
high-heeled, buckled boots. His blue eyes were light, bright, and
sparkling behind half-moon spectacles and his nose was very long
and crooked, as though it had been broken at least twice. This
man’s name was Albus Dumbledore.

Albus Dumbledore didn’t seem to realize that he had just
arrived in a street where everything from his name to his boots was
unwelcome. He was busy rummaging in his cloak, looking for
something. But he did seem to realize he was being watched, because
he looked up suddenly at the cat, which was still staring at him
from the other end of the street. For some reason, the sight of the
cat seemed to amuse him. He chuckled and muttered, “I should
have known.”

He found what he was looking for in his inside pocket. It seemed
to be a silver cigarette lighter. He flicked it open, held it up in
the air, and clicked it. The nearest street lamp went out with a
little pop. He clicked it again — the next lamp flickered
into darkness. Twelve times he clicked the Put-Outer, until the
only lights left on the whole street were two tiny pinpricks in the
distance, which were the eyes of the cat watching him. If anyone
looked out of their window now, even beady-eyed Mrs. Dursley, they
wouldn’t be able to see anything that was happening down on
the pavement. Dumbledore slipped the Put-Outer back inside his
cloak and set off down the street toward number four, where he sat
down on the wall next to the cat. He didn’t look at it, but
after a moment he spoke to it.

“Fancy seeing you here,
Professor McGonagall.”

He turned to smile at the tabby, but it had gone. Instead he was
smiling at a rather severe-looking woman who was wearing square
glasses exactly the shape of the markings the cat had had around
its eyes. She, too, was wearing a cloak, an emerald one. Her black
hair was drawn into a tight bun. She looked distinctly r
"""

## 📕 Procesar un documento completo

### 🪓 Dividir documentos

In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 2000,
    chunk_overlap  = 200
)

book_path = "potter_books/processed/hp_01/chapter_01.md"
chapter = UnstructuredMarkdownLoader(book_path).load()
chunks = text_splitter.split_documents(chapter)

### 🛠️ + 🖇️ Procesar y juntar

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce")
result = chain.run(chunks[0:2])

In [ ]:
print(result)

### 🔬 Examinar el proceso por dentro

#### Map-Reduce
```
        📕{documento}
       /            \
  📜{cacho_0} .. 📜{cacho_N}
      |                |
🔗{map_chain} .. 🔗{map_chain}
      \              /
     🔗{reduce_chain}
             |
     💬{final_output}
```

##### Specialized classes

In [ ]:
from langchain.chains import LLMChain, ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# short summary of each section of the book
map_template = """
The following is a set of paragraph extracted from the chapter {chapter} of the book {book_title}:
{docs}
Based on this list of docs, please summarize them. Be brief 20, words maximun.
"""
map_prompt = llama_prompt(system_prompt, map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt, output_parser=StrOutputParser())

# Join several summaries into a single a single bigger summary
reduce_template = """
The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary. Be brief, 250 words maximun.
"""
reduce_prompt =  llama_prompt(system_prompt, reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt, output_parser=StrOutputParser())

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,            
    token_max=4000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs",
    return_intermediate_steps=False,
)

response = map_reduce_chain.run(
    input_documents=chunks[0:2], 
    book_title="Harry Potter and the Philosopher's Stone",
    chapter="1"
)

##### LCEL

In [ ]:
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable.base import RunnableEach, RunnableSequence
from langchain.schema.runnable import RunnablePassthrough

map_template = """
The following is a set of paragraph extracted from the chapter {chapter} of the book {book_title}:
{docs}
Based on this list of docs, please summarize them. Be brief 20, words maximun.
"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = (
    {"doc": RunnablePassthrough()}
    | map_prompt
    | llm
    | StrOutputParser()
)

reduce_template = """
The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary. Be brief, 250 words maximun.
"""
reduce_prompt = PromptTemplate.from_template(template=reduce_template)
reduce_chain = (
    {"docs": RunnablePassthrough()}
    | reduce_prompt
    | llm
    | StrOutputParser()
)
map_reduce = (
    RunnableEach(bound=map_chain)
    | reduce_chain
)

In [ ]:
response

#### Refine
```
📕{documento}
 | \
 | 📜{cacho_0}-🔗{initial}-💬{partial_output_0} 
 \                  _________________/
  \                |
   📜{cacho_1}-🔗{refine}-💬{partial_output_1} 
```

##### Specialized Classes

In [ ]:
from langchain.chains import LLMChain, RefineDocumentsChain

document_variable_name = "docs"
initial_response_name = "prev_response"

# Initial chain
initial_prompt = """
The following is a paragraph extracted from the chapter {chapter} of the book {book_title}:
{docs}
Based on this paragraph, please summarize them. Be brief 250, words maximun.
"""
initial_prompt = llama_prompt(system_prompt, initial_prompt)
initial_chain = LLMChain(llm=llm, prompt=initial_prompt, output_parser=StrOutputParser())

# Refine chain
refine_prompt = """
Here is the summary of the chapter {chapter} of the book {book_title} done so far:
{prev_response}
Given the next paragraph, refine the summary. Be brief 250, words maximun.
{docs}
"""
refine_prompt = llama_prompt(system_prompt, refine_prompt)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt, output_parser=StrOutputParser())

# how to get the information of a chunk
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

# do all the process
refine_document_chain = RefineDocumentsChain(
    initial_llm_chain=initial_chain,
    refine_llm_chain=refine_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name
)

response = refine_document_chain.run(
    input_documents=chunks[0:2], 
    book_title="Harry Potter and the Philosopher's Stone",
    chapter="1"
)

In [ ]:
print(response)

##### LCEL

In [ ]:
from langchain.schema.runnable.base import RunnableEach, RunnableSequence, RunnableLambda

def create_first_chain():
    first_template = """
    Summarize the next paragraph. Be brief 10 words or less.\n{doc}
    """
    first_prompt = PromptTemplate.from_template(template=first_template)
    first_chain = (
        {"doc": RunnablePassthrough()}
        | first_prompt
        | llm
        |{'summary': RunnablePassthrough()}
    )
    return first_chain

def create_middle_chain(chunks):
    def _create_middle_item_chain(document):
        refine_template = """
        Given the next summary with the information processed so far:
        {summary}
        Update it with the next paragraph, be brief 10 words or less:
        {paragraph}
        """
        middle_prompt = PromptTemplate.from_template(template=refine_template, partial_variables={'paragraph': document.page_content})
        return (
            middle_prompt
            | llm
            |{'summary': RunnablePassthrough()}
        )
        
    return [_create_middle_item_chain(d) for d in chunks]
    
def create_refine_chain(chunks):
    chain = RunnableSequence(
        first = create_first_chain(),
        middle = create_middle_chain(chunks),
        last = RunnableLambda(lambda x: x)
    )
    return chain

create_refine_chain(chunks[1:3]).invoke(chunks[0])

## 📘 Interactuar con un documento

### 🪓 Dividir el documento 

In [ ]:
from os import listdir
from os.path import isfile, join
from functools import reduce
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1500,
    chunk_overlap  = 150
)

book_path = "potter_books/processed/hp_01"
chapters = [join(book_path, f) for f in listdir(book_path) if isfile(join(book_path, f)) and f.endswith(".md")]
chapters = reduce(lambda x,y: x + UnstructuredMarkdownLoader(file_path=y).load(), chapters, []) 
chunks = text_splitter.split_documents(chapters)

len(chunks)

### ️🗄️ Generar la base de datos

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings_model
)

### ️⚒️ Extraer cachos relevantes

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

question = "Who is Harry's best friend?"

docs_chain = RunnablePassthrough() | vectordb.as_retriever(search_kwargs = {'k' : 2})

for doc in docs_chain.invoke(question):
    print("=====================================")
    print(doc.page_content)
    print("=====================================")

### 🏗️ Procesar documentos

In [ ]:
from langchain.chains import RetrievalQA

qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs = {'k' : 2}),
    chain_type="stuff"
)

question = "What Hogwarts house is Harry in? Only tell the name of the house"
result = qa_chain_mr({"query": question})

print(question)
print(f">> {result['result']}")

### 🔬 Examinar el proceso por dentro

#### Replicar proceso

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain_mr = (
    {"context": RunnablePassthrough() | vectordb.as_retriever(search_kwargs = {'k' : 2}), "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser()
)

question = "What Hogwarts house is Harry in? Only tell the name of the house"
result = qa_chain_mr.invoke(question)

print(question)
print(f">> {result}")

#### Examinar prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain_mr = (
    {"context": RunnablePassthrough() | vectordb.as_retriever(search_kwargs = {'k' : 11}), "question": RunnablePassthrough()} 
    | prompt
)

qa_prompt_response = qa_chain_mr.invoke(question)
print(qa_prompt_response.messages[0].content)
print(count_tokens(qa_prompt_response.messages[0].content, llm))

#### Reducir el documento a algo manejable

In [ ]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain, ReduceDocumentsChain

def make_reduce_chain():

    # Join several summaries into a single a single bigger summary
    reduce_template = """
    The following is set of paragraphs from a Harry Potter book:
    {docs}
    Take these and use your knowledge about the Harry Potter universe to summarize the main points. Be brief, 250 words maximun. Focus on the information that is relevant to the next question:
    {question}
    """
    reduce_prompt =  llama_prompt(system_prompt, reduce_template)
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt, output_parser=StrOutputParser())

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="docs"
    )

    # Combines and iteravely reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        combine_documents_chain=combine_documents_chain,
        collapse_documents_chain=combine_documents_chain,            
        token_max=500,
    )

    reduce_question_chain = (
        {"input_documents": itemgetter("question") | vectordb.as_retriever(search_kwargs = {'k' : 2}), "question": itemgetter("question")}
        | reduce_documents_chain
        | (lambda x: x['output_text'])
    )

    return reduce_question_chain

# question answer chain
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
qa_prompt = ChatPromptTemplate.from_template(template)

qa_chain_mr = (
    {"context": make_reduce_chain(), "question": itemgetter("question")}
    | qa_prompt 
    | llm
    | StrOutputParser()
)

question = "What Hogwarts house is Harry in? Only tell the name of the house"
result = qa_chain_mr.invoke({'question':question})

print(question)
print(f">> {result}")

# 🧠 Memoria

## ️🗄️ Generar la base de datos

In [ ]:
from functools import reduce
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from os import listdir
from os.path import isfile, join

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 150
)

book_path = "potter_books/processed/hp_01"
chapters = [join(book_path, f) for f in listdir(book_path) if isfile(join(book_path, f)) and f.endswith(".md")]
chapters = reduce(lambda x,y: x + UnstructuredMarkdownLoader(file_path=y).load(), chapters, []) 
chunks = text_splitter.split_documents(chapters)

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings_model
)

## 👎🧠Sin memoria

In [ ]:
from langchain.chains import RetrievalQA

qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs = {'k' : 2}),
    chain_type="stuff"
)

question = "What Hogwarts house is Harry in? Only tell the name of the house"
result = qa_chain_mr({"query": question})

print(question)
print(f">> {result['result']}")

question = "Tell me three other people in that house? Only tell their full names"
result = qa_chain_mr({"query": question})

print(question)
print(f">> {result['result']}")

## 👍🧠 Con memoria

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs = {'k' : 2}),
    chain_type="stuff",
    memory=memory
)

question = "What Hogwarts house is Harry in? Only tell the name of the house"

print(question)
result = conversational_chain({"question": question})
print(f">> {result['answer']}")

question = "Tell me three other people in that house? Only tell their full names"

print(question)
result = conversational_chain({"question": question})
print(f">> {result['answer']}")

## 🔬 Examinar el proceso por dentro

#### Transformar preguntas:
Reformular una pregunta dado el contexto

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

def transform_question_chain(memory):
    
    text_prepare_question = """
    Given the following conversation and a follow up question rephrase the follow up question to be a standalone question, in its original language. 
    Be brief, never propose more than one question. Never continue the  conversation
    {chat_history}
    Follow Up Input: {question}
    Standalone Question:
    """
    prompt_prepare_question = llama_prompt(system_prompt, text_prepare_question)

    chain_prepare_question = (
        {"chat_history": lambda x: memory.load_memory_variables({})["chat_history"], "question": RunnablePassthrough()}
        | prompt_prepare_question
        | llm
        | StrOutputParser()
    )
    return chain_prepare_question

memory = ConversationBufferMemory(memory_key="chat_history")
memory.save_context(
    {'input': 'What Hogwarts house is Harry in? Only tell the name of the house'},
    {'output': 'Gryffindor'}
)

transform_question_chain(memory).invoke("Tell me three other people in that house? Only tell their full names")

#### Responder la pregunta modificada

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.save_context(
    {'input': 'What Hogwarts house is Harry in? Only tell the name of the house'},
    {'output': 'Gryffindor'}
)

text_solve_question = """
User your knowledge about the Harry Potter universe to answer the following question:
QUESTION: {question}
USEFULL ANSWER:
"""
prompt_solve_question = llama_prompt(system_prompt,text_solve_question)

conversational_chain = (
    {"question": transform_question_chain(memory)}
    | prompt_solve_question
    | llm
    | StrOutputParser()
)
print(conversational_chain.invoke("Tell me three other people in that house? Be brief, only tell their full names"))

#### Actualizar la memoria cada llamada

In [ ]:
from langchain.chains import LLMChain

def print_memory(memory, params={}):
    print("MEMORY BUFFER==>:")
    print(memory.load_memory_variables(params)["chat_history"])
    print("<==============")

def ask_question(chain, question):
    print(f"Human: {question}")
    print(f">> AI: {chain.invoke(question)}")    

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)

solve_question_chain = LLMChain(
    prompt=prompt_solve_question,
    llm=llm,
    memory=memory
)
conversational_chain = (
    {"question": RunnablePassthrough()}
    | solve_question_chain
    | (lambda x: x['text'])
)

print_memory(memory)

ask_question(conversational_chain, "What Hogwarts house is Harry in? Only tell the name of the house")

print_memory(memory)

ask_question(conversational_chain, "Tell me three other people in that house? Only tell their full names")

print_memory(memory)

## 🗓️ Tipos de memoria

### Window Buffer

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

print_memory(memory)

memory.save_context(
    {'input': 'What Hogwarts house is Harry in? Only tell the name of the house'},
    {'output': 'Gryffindor'}
)

print_memory(memory)

memory.save_context(
    {'input': 'Tell me three other people in that house? Only tell their full names'},
    {'output': 'Sure! Here are three other people in the house with Harry Potter:\n1. Ronald Bilius Weasley\n2. Hermione Jean Granger\n3. Frederick Gideon Lester'}
)

print_memory(memory)

memory.save_context(
    {'input': 'Who is Frederick Gideon Lester?'},
    {'output': 'Who knows'}
)

print_memory(memory)

### Conversation Summary Buffer

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

def make_summary_buffer():
    memory_system_prompt = """
    You are a helpfull AI in charge of summarizing conversations between a human and an AI model. Please do the summary exactly as the user tells you to do so.
    """
    memory_user_prompt = """
    Given the next summary of the conversation so far: 
    {summary}
    Use the new lines of conversations to update the summary with new conversation. Be brief, only responde with the new summary, make it shorter than 20 words. Never include information that do not appear in the new lines or the previous summary
    {new_lines}
    """
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, memory_key="chat_history", prompt=llama_prompt(memory_system_prompt, memory_user_prompt))
    return memory

memory = make_summary_buffer()

print_memory(memory)

memory.save_context(
    {'input': 'What Hogwarts house is Harry in? Only tell the name of the house'},
    {'output': 'Gryffindor'}
)

print_memory(memory)

memory.save_context(
    {'input': 'Tell me three other people in that house? Only tell their full names'},
    {'output': 'Sure! Here are three other people in the house with Harry Potter:\n1. Ronald Bilius Weasley\n2. Hermione Jean Granger\n3. Frederick Gideon Lester'}
)

print_memory(memory)

memory.save_context(
    {'input': 'Who is Frederick Gideon Lester? Be brief no more than 20 words'},
    {'output': "Frederick Gideon Lester is a Hogwarts student in Harry's year."}
)

print_memory(memory)

### Vector Store

In [ ]:
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.memory import VectorStoreRetrieverMemory

embedding_size = 384
index = faiss.IndexFlatL2(embedding_size)
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
memory_vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

retriever = memory_vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever,  memory_key="chat_history")

memory.save_context(
    {'input': 'What Hogwarts house is Harry in? Only tell the name of the house'},
    {'output': 'Gryffindor'}
)
memory.save_context(
    {'input': 'Tell me three other people in that house? Only tell their full names'},
    {'output': 'Sure! Here are three other people in the house with Harry Potter:\n1. Ronald Bilius Weasley\n2. Hermione Jean Granger\n3. Frederick Gideon Lester'}
)
memory.save_context(
    {'input': 'Who is Frederick Gideon Lester? Be brief no more than 20 words'},
    {'output': "Frederick Gideon Lester is a Hogwarts student in Harry's year."}
)

print_memory(memory, {"prompt": "What is Frederick surename?"})

# 🤖 Agentes

## 🦙🤖📜 LLama Agent Sources

### 🦙🤖🦜 LLama Agent Parser

In [ ]:
import json
import re
from langchain.chains import LLMChain
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from pydantic import PrivateAttr
from typing import Union

class CustomAgentParser(AgentOutputParser):
    _chain: LLMChain = PrivateAttr(True)
    
    def __init__(self, expected_format: str, llm):
        super().__init__()
        
        system_prompt = "Your are a JSON expert. Your task is to correct a JSON document that is formated wrong. Act as a code generator, and always include in your response a version of JSON document corrected."
        human_prompt = """
            Here is a piece of JSON code with some errors: 
            {error_json}
            Please use your JSON kwonledge to fix it. The format that the document should follow is this, do not fix this format is only for reference:
            {format}
            
            Fix the document in two steps:
            Step 1: Indicate the errors in the JSON document. For example, missing closing quotes or arrays or documents. Be brief, no more than 20 words.
            Step2: Build a JSON document with the errors fixed. Always return the document enclosed in the tags <json> and </json>
        """
        prompt = llama_prompt(system_prompt, human_prompt,format=expected_format)
        chain = {"error_json": RunnablePassthrough()} | prompt | llm | ExtractJsonOutputParser()
        self._chain = chain

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        
        final_answer = self._final_answer(text)
        if not final_answer is None:
            return AgentFinish(
                return_values={"output": final_answer}, 
                log=text
            )

        action_str = self._action(text)
        if not action_str is None:
            action = self._parse_json(action_str)
            input = {arg['keyword']: arg['value'] for arg in action['inputs']}
            return AgentAction(
                log=self._clean_observations_from_log(text),
                tool=action['name'], 
                tool_input=input
            )    
        
        raise ValueError("Wrong format, not an action or an answer")

    def _final_answer(self, text:str) -> Union[None,str]:
        pattern = r"#Final Answer:(.*?)$"
        match = re.search(pattern, text, re.DOTALL)

        answer = match.group(1).strip() if match else None
        return answer

    def _action(self, text:str) -> Union[None,str]:
        # Define the regular expression pattern
        pattern = r"#Action:(.*?)(?=#Observation:|$)"
        
        # Use re.findall to extract the text between the tags
        matches = re.findall(pattern, text, re.DOTALL)
        
        # Clean up the extracted text (remove leading/trailing whitespace)
        cleaned_matches = [match.strip() for match in matches]

        return cleaned_matches[-1] if len(cleaned_matches) > 0 else None

    def _parse_json(self, text: str):
        json_regex = re.compile(r'<json>(.*)</json>', re.DOTALL)
        
        documents = json_regex.findall(text)
        if len(documents) == 0:
            return self._fix_json_errors(text)
        
        json_content = documents[0]
        try:
            return json.loads(json_content)
        except json.JSONDecodeError:
            return self._fix_json_errors(json_content)

    def _clean_observations_from_log(self, text:str):
        pattern = r"(?s)(#Observation:[^\n]*$)"
        text_without_observation = re.sub(pattern, "", text)
        return text_without_observation

    def _fix_json_errors(self, json_text: str):
            return json.loads(self._chain.invoke(json_text))
        

### 🦙🤖💬 LLama Agent Prompt

In [ ]:
from langchain.chains import LLMChain
from langchain.agents import LLMSingleActionAgent
from langchain.tools.render import format_tool_to_openai_function

def format_scratchpad(steps):
    scratchpad = ""
    for action, value in steps:
        scratchpad += f"{action.log}\n[INST]#Observation:{value}[/INST]\n"
    return scratchpad

def toolkit_prompt(toolkit):
    tools_dict = {
        "tools": [format_tool_to_openai_function(t) for t in toolkit]
    }
    # return a string with the json
    return f"<json>{json.dumps(tools_dict,indent=2)}</json>"

def create_llama_agent(toolkit):
    system_prompt = """
    <<SYS>>
    You are a helpful AI asistant capable of reasoning and using tools. Answer the question the best you can.
    Explain every decission you take, be brief do it in as few words as possible, 50 maximum.
    When neccesary use the tools indicated by the user to answer the question. 
    If there is no tool that helps you answering the question, try answering the question yourself or indicate that you do not know if you lack the information to answer it. 
    Only use the tools to do the actions they were designed for, never apply a tool to do a function they were not designed for.
    You never know the result of using a tool. Never try to guess that result. The results of using a tool will be inputed by the user. 
    Only use one tool at a time. When using a tool, stop and wait for the user to input the "#Observation:" value. 
    You are not allowed to write "#Observation:", that keyword is reserved for the user.
    Always start your intervition with a new thought using the keyword "#Thought".
    <</SYS>>
    """
    
    human_prompt = """
    [INST] Your task is to answer the question posed by the user. When neccesary, use the next set of tools to answer the question, please use the descriptions to know when to select a tool. 
    Never use a tool that is not included in the document or for a purpose other than the one stated in its description:
    {tools}
    
    To use a tool please follow the next JSON sintax:
    {action_example}
    
    You must always follow the next steps to aswer a question. You may need to use several tools to answer the question or no tool at all. Never try to use other tools outside the ones provided. Never use a tool for things outside their description.  
    In case no suitable tool is available for answering the question, skip the #Action and #Observation steps, go to the '#Final Answer' and try to answer the question yourself.
    
    #Question: the input question you must answer
    #Thought: List the tools that could be use to answer the question and decide what tool to use next if any. Always explain why the tools are suitable and why not.
    #Action: Only if there is suitable tool to answer the question, indicate which tool to use. Always finish the action with a JSON document enclosed between <json> and </json> tags. Wait for user input
    <json>....</json>
    #Observation: The user will input the result of executing the desired action. Never write the #Observation, wait for the user input.
    #Thought: Analize the past chain of thoughts and the previous observation to decide what to do next. If more tools are necesary, do the next call. If no more tools are neccesary, answer the question.
    #Action: If necessary, call the next tool to be used. Wait for user input
    #Observation: 
    ... (repear N times)
    #Action:
    #Observation:
    #Thought: Check that you have all the information neccesary to answer the question and no more tools are pending to be called. If so, you can answer the question.
    #Final Answer: the final answer to the original input question. 
    
    #### BEGIN THE TASK #####
    
    #Question: {input} [/INST]
    {agent_scratchpad}
    #Thought: 
    """
    
    action_example = """
    <json>
    {{
        "type": "action",
        "name": string, #one of the next values {tool_names}
        "inputs": Array of {{
            "keyword": string,
            "value": Any
        }}
    }}
    </json>
    """
    action_example = action_example.format(tool_names=[t.name for t in toolkit])

    prompt = PromptTemplate.from_template(
        template=system_prompt+"\n"+human_prompt,
        partial_variables={
            "tools":toolkit_prompt(toolkit), 
            "action_example":action_example
        }
    )
    parser = CustomAgentParser(
        expected_format=action_example,
        llm=llm
    )        
    
    agent =  (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_scratchpad(x['intermediate_steps'])
        } | prompt | llm | parser
    )
    return agent

## 🤖⚙️ Agent Running Loop

In [ ]:
import json
from langchain.agents import tool
from langchain.agents import AgentExecutor

@tool
def get_word_length(word: str) -> int:
    """
    Returns the length of a word.
    """
    return len(word)

@tool
def pow(number:float, N:int) -> float:
    """
    Returns the number to the power N
    """
    return number**N

toolkit = [get_word_length, pow]

agent = create_llama_agent(toolkit)
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True, max_iterations=6)

agent_executor.invoke({
    #"input": "how many letters in the word 'educa'?"
    "input": "Please translate the next sentence into emojis: 'I have cooked pig ribs for dinner'"
    #"input": "What is 2 to the power of 3?"
    #"input": "Count the number of letter in the word 'luna' and raise the result to the power of 3"
})